In [1]:
!pip install mesa seaborn --quiet

In [2]:
# Importamos las clases que se requieren para manejar los agentes (Agent) y su entorno (Model).
# Cada modelo puede contener múltiples agentes.
from mesa import Agent, Model 

# Debido a que necesitamos que existe un solo agente por celda, elegimos ''SingleGrid''.
from mesa.space import SingleGrid

# Con ''SimultaneousActivation, hacemos que todos los agentes se activen ''al mismo tiempo''.
from mesa.time import SimultaneousActivation

# Haremos uso de ''DataCollector'' para obtener información de cada paso de la simulación.
from mesa.datacollection import DataCollector

# matplotlib lo usaremos crear una animación de cada uno de los pasos del modelo.
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.animation as animation
plt.rcParams["animation.html"] = "jshtml"
matplotlib.rcParams['animation.embed_limit'] = 2**128

# Importamos los siguientes paquetes para el mejor manejo de valores numéricos.
import numpy as np
import pandas as pd

# Definimos otros paquetes que vamos a usar para medir el tiempo de ejecución de nuestro algoritmo.
import time
import datetime

In [ ]:
class LuigiAgent(Agent):
  def __init__(self, unique_id, model):
    super().__init__(unique_id, model)
    self.points = 4

  
  def move(self):
    possible_moves = self.model.grid.get_neighborhood(self.pos, moore=True, include_center=False)
    new_position = self.random.choice(possible_moves)
    if self.model.grid.is_cell_empty(new_position):
        self.model.grid.move_agent(self, new_position)
      
  def interact(self):
    # Revisa si hay un punto de interés o un fantasma
    cellmates = self.model.grid.get_cell_list_contents([self.pos])
    for agent in cellmates:
      if isinstance(agent, GreenGhostAgent):
        self.perform_exorcism(agent)
      elif isinstance(agent, ArtifactAgent):
        self.examine_artifact(agent)

  def perform_knocked(self, ghost):
    if isinstance(ghost, GreenGhostAgent):
      #Define lógica para noquear a un fantasma de la casilla
      ghost.knocked = True
      print(f"Luigi {self.unique_id} ha noqueado un fantasma en {self.pos}")

  def perform_exorcism(self, ghost):
    if isinstance(ghost, GreenGhostAgent):
      #Define lógica para eliminar a un fantasma de la casilla
      ghost.absorb = True
      print(f"Luigi {self.unique_id} ha absorbido un fantasma en {self.pos}")

  def examine_artifact(self, artifact):
    if isinstance(artifact, ArtifactAgent):
      # Define la lógica para examinar el artefacto
      artifact.recovered = True
      print(f"Luigi {self.unique_id} ha recuperado un artefacto en {self.pos}")

  def step(self):
    self.move()
    self.interact()

In [ ]:
class GreenGhostAgent(Agent):
  def __init__(self, unique_id, model):
    super().__init__(unique_id, model)

In [ ]:
class RedGhostAgent(Agent):
  def __init__(self, unique_id, model):
    super().__init__(unique_id, model)

In [ ]:
class ArtifactAgent(Agent):
  def __init__(self, unique_id, model):
    super().__init__(unique_id, model)

In [ ]:
class MansionModel(Model):
    def __init__(self, width, height, num_agents):
        self.num_agents = num_agents
        self.grid = SingleGrid(width, height, True)
        self.schedule = SimultaneousActivation(self)
        self.datacollector = DataCollector(model_reporters={"Damage": lambda m: m.total_damage})

        for i in range(self.num_agents):
            luigi = LuigiAgent(i, self)
            self.schedule.add(luigi)
            x = self.random.randrange(self.grid.width)
            y = self.random.randrange(self.grid.height)
            self.grid.place_agent(luigi, (x, y))

        self.total_damage = 0 #Este es el da;o de la casa 

    def step(self):
        self.schedule.step()
        self.datacollector.collect(self)